In [1]:
QRELS_PATH = 'evaluation/qrels.json'

In [2]:
SEED = 2024
ANALYSIS_COL = 'userAcceleration.z'
PRECOMPUTED_DISTANCES = f"evaluation/dtw/{ANALYSIS_COL.replace('.', '-')}_distances.npy"
RUN_FILE = f"evaluation/runs/dtw_{ANALYSIS_COL.replace('.', '-')}.json"

# DIST = 'adtw'
# PRECOMPUTED_DISTANCES = f"evaluation/multivar/{DIST}_distances.npy"
# RUN_FILE = f"evaluation/runs/{DIST}_multivar.json"

# Preparando avaliação

In [3]:
from ranx import Qrels

In [4]:
qrels = Qrels.from_file(QRELS_PATH)

# Executando a busca por similaridade

In [5]:
from ranx import Run
import numpy as np
from aeon.distances import dtw_pairwise_distance

In [6]:
def compute_neighbors_by_similarity(
    distances: np.array,
    topk: int = 100) -> dict:

  run_dict = {}

  for qid in range(distances.shape[0]):
    all_sims = distances[qid]
    topk = np.argsort(all_sims)

    # ignorando o proprio exemplo de busca
    estimated_ids = topk[1:]
    scores = list(range(len(estimated_ids)))[::-1]

    run_dict[str(qid)] = {str(n): float(score) for n, score in zip(estimated_ids, scores)}

  return run_dict

In [7]:
PRECOMPUTED_DISTANCES

'evaluation/dtw/userAcceleration-z_distances.npy'

In [8]:
distances = np.load(PRECOMPUTED_DISTANCES)

In [9]:
run_dict = compute_neighbors_by_similarity(distances)

# Avaliando métricas de RI

In [10]:
from ranx import evaluate

In [11]:
METRICS = ["recall@10", 'map@10', 'hit_rate@10', 'mrr@10']
# METRICS = 'mrr@10'

In [12]:
run = Run(run_dict)

In [13]:
metrics = evaluate(qrels, run, METRICS)
metrics

{'recall@10': 0.20341435185185186,
 'map@10': 0.16543990667254557,
 'hit_rate@10': 0.8819444444444444,
 'mrr@10': 0.732771164021164}

# Serializando Run

In [14]:
RUN_FILE

'evaluation/runs/dtw_userAcceleration-z.json'

In [15]:
run.save(RUN_FILE)